In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from cassandra.cluster import Cluster

In [10]:
spark = SparkSession.builder \
    .appName("ElhubSilver") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

✅ SparkSession started with Cassandra integration


# Production

In [3]:
bronze_df = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_raw", keyspace="elhub_data")
    .load()
)

In [4]:
silver_df = (
    bronze_df
    .select(
        col("priceArea").alias("pricearea"),
        col("productionGroup").alias("productiongroup"),
        col("startTime").alias("starttime"),
        col("quantityKwh").alias("quantitykwh")
    )
    .withColumn("starttime", col("starttime").cast("timestamp"))
    .withColumn("quantitykwh", col("quantitykwh").cast("double"))
)

In [5]:
cluster = Cluster(["localhost"])
session = cluster.connect("elhub_data")
session.execute("TRUNCATE production_silver")

In [6]:
(
    silver_df
    .write
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .mode("append")  
    .save()
)
print("✅ Data written to elhub_data.production_silver (after truncate)")

✅ Data written to elhub_data.production_silver (after truncate)


In [7]:
# Check for missing values
missing_values = silver_df.filter(
    col("pricearea").isNull() |
    col("productiongroup").isNull() |
    col("starttime").isNull() |
    col("quantitykwh").isNull()
).count()
if missing_values == 0:
    print("✅ No missing values in silver_df")
else:
    print(f"⚠️ There are {missing_values} rows with missing values in silver_df")

✅ No missing values in silver_df


In [8]:
# Describe the dataframe
silver_df.describe().show()

+-------+---------+---------------+------------------+
|summary|pricearea|productiongroup|       quantitykwh|
+-------+---------+---------------+------------------+
|  count|   871153|         871153|            871153|
|   mean|     NULL|           NULL| 704176.0272003636|
| stddev|     NULL|           NULL|1459343.4750884627|
|    min|      NO1|          hydro|               0.0|
|    max|      NO5|           wind|       1.0054233E7|
+-------+---------+---------------+------------------+



# Consumption

In [11]:
bronze_df_con = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="consumption_raw", keyspace="elhub_data")
    .load()
)

In [12]:
silver_df_con = (
    bronze_df_con
    .select(
        col("pricearea").alias("pricearea"),
        col("consumptiongroup").alias("consumptiongroup"),
        col("starttime").alias("starttime"),
        col("endtime").alias("endtime"),
        col("quantitykwh").alias("quantitykwh")
    )
    .withColumn("starttime", col("starttime").cast("timestamp"))
    .withColumn("endtime", col("endtime").cast("timestamp"))
    .withColumn("quantitykwh", col("quantitykwh").cast("double"))
)

In [16]:

cluster = Cluster(["localhost"])
session = cluster.connect("elhub_data")

session.execute("""
CREATE TABLE IF NOT EXISTS consumption_silver (
    pricearea TEXT,
    consumptiongroup TEXT,
    starttime TIMESTAMP,
    endtime TIMESTAMP,
    quantitykwh DOUBLE,
    PRIMARY KEY ((pricearea), consumptiongroup, starttime)
)
""")

print("✔ consumption_silver table created or verified.")

✔ consumption_silver table created or verified.


In [ ]:
cluster = Cluster(["localhost"])
session = cluster.connect("elhub_data")
session.execute("TRUNCATE consumption_silver")

In [17]:
(
    silver_df_con
    .write
    .format("org.apache.spark.sql.cassandra")
    .options(table="consumption_silver", keyspace="elhub_data")
    .mode("append")  
    .save()
)
print("✅ Data written to elhub_data.consumption_silver (after truncate)")

✅ Data written to elhub_data.consumption_silver (after truncate)


In [18]:
missing_values = silver_df_con.filter(
    col("pricearea").isNull() |
    col("consumptiongroup").isNull() |
    col("starttime").isNull() |
    col("endtime").isNull() |
    col("quantitykwh").isNull()
).count()

if missing_values == 0:
    print("✔ No missing values in silver_df_con")
else:
    print(f"⚠️ {missing_values} rows in silver_df_con have missing values")

✔ No missing values in silver_df_con


In [19]:
# Describe the dataframe
silver_df_con.describe().show()

+-------+---------+----------------+-----------------+
|summary|pricearea|consumptiongroup|      quantitykwh|
+-------+---------+----------------+-----------------+
|  count|   874800|          874800|           874800|
|   mean|     NULL|            NULL|589486.6804404076|
| stddev|     NULL|            NULL|635637.7736697067|
|    min|      NO1|           cabin|         8731.135|
|    max|      NO5|        tertiary|        4264747.0|
+-------+---------+----------------+-----------------+

